# Retrieve french box office for the last 10 years from Wikipedia

In this notebook we are going to create a database with the movies that had the highest number of entries in french movie theaters over the last 10 years. Our source are the tables in the Wikipedia pages called Box-office_France_YYYY (for example
https://fr.wikipedia.org/wiki/Box-office_France_2022)

In [1]:
#import librairies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

We create a function to retrieve the information from the tables in the wikipedia pages

In [2]:
def get_bo_tables(url, num):
    
    '''extract the box office tables from a wikipedia page
    
    Inputs
    -------
    url : str
    the wikipedia page we want to extract tables from
    num : integer
    number of tables we want to extract from the page
    
    Output
    ------
    movie_dic_list : list of dictionnaries
    list of dictionnaries containing the rows of of each table
    
    '''

    # Make a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the table element on the page
    tables = soup.find_all('table')
    
    #create empty list to store the dictionnaries
    movie_dic_list=[]
    
    # Iterate over the tables and extract the data
    for table in tables[:num]:

        #create empty dictionnary
        movie_dic={}

        # Extract the rows of the table
        rows = table.find_all('tr')

        # Iterate over the rows and extract the data
        for i,row in enumerate(rows):
            # Find the cells in the row
            cells = row.find_all('td')

            # Extract the text from each cell
            data = [cell.text.strip() for cell in cells]

            # create entry in the dictionnary       
            movie_dic[i]=data

        if movie_dic[0]==[]:
            del movie_dic[0]
        
        movie_dic_list.append(movie_dic)
    
    return movie_dic_list

We use our function to retrieve all the box office tables from 2013 to 2022 (except 2020) and create a dataframe with all the movies. In 2020 we will need two tables so we treat this year separately.

In [5]:
bo_df=pd.DataFrame()

for year in list(range(2013,2020))+ [2021, 2022]:
    
    url='https://fr.wikipedia.org/wiki/Box-office_France_'+str(year)
    movie_dic=get_bo_tables(url, num=1)
    bo=pd.DataFrame.from_dict(movie_dic[0], orient='index').replace('', np.nan).dropna(axis=1, how='all')
    if year != 2022:
        bo.reset_index(inplace=True)
    if year < 2020 :
        bo.drop(bo.columns[-1],axis=1, inplace=True) #remove weeks columns
    bo.columns=['rank','movie', 'director', 'entries'] #rename columns
    bo['year']=year
    bo_df=pd.concat([bo_df,bo])

bo_df['entries'].str.replace(' ','').astype(int) 
bo_df.reset_index(drop=True, inplace=True)

In [6]:
bo_df.head()

,rank,movie,director,entries,year
0,1,La Reine des neiges,Chris Buck et Jennifer Lee,5 149 518,2013
1,2,Le Hobbit : La Désolation de Smaug,Peter Jackson,4 701 246,2013
2,3,"Moi, moche et méchant 2",Pierre Coffin et Chris Renaud,4 655 036,2013
3,4,Iron Man 3,Shane Black,4 386 939,2013
4,5,Django Unchained,Quentin Tarantino,4 303 569,2013


For 2020 we need to retrieve 2 tables from the wikipidia page and concatenate them.

In [234]:
url_2020 = 'https://fr.wikipedia.org/wiki/Box-office_France_2020'
movie_dic=get_bo_tables(url_2020, num=2)

In [251]:
bo_2020=pd.DataFrame()
for i in range(2):
    bo=pd.DataFrame.from_dict(movie_dic[i], orient='index').replace('', np.nan).dropna(axis=1, how='all')
    bo.columns=['movie', 'director', 'entries']
    bo['year']=2020
    bo_2020=pd.concat([bo_2020,bo], ignore_index=True)
    
bo_2020['rank']=bo_2020['entries'].str.replace(' ','').astype(int).rank(ascending=False) #create rank column
bo_2020['year']=2020

In [292]:
#create final dataframe
bo_df=pd.concat([bo_df, bo_2020[['rank','movie', 'director', 'entries','year']]]) 

In [9]:
bo_df.head()

,rank,movie,director,entries,year
0,1,La Reine des neiges,Chris Buck et Jennifer Lee,5 149 518,2013
1,2,Le Hobbit : La Désolation de Smaug,Peter Jackson,4 701 246,2013
2,3,"Moi, moche et méchant 2",Pierre Coffin et Chris Renaud,4 655 036,2013
3,4,Iron Man 3,Shane Black,4 386 939,2013
4,5,Django Unchained,Quentin Tarantino,4 303 569,2013


In [8]:
bo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rank      414 non-null    object
 1   movie     414 non-null    object
 2   director  414 non-null    object
 3   entries   414 non-null    object
 4   year      414 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 16.3+ KB


In [10]:
#save data 
bo_df.to_csv("box_office_20132022.csv")